In [1]:
import pandas as pd
import numpy as np
import datetime
import subprocess
from filegenerators import *
pd.options.display.float_format = '{:.2e}'.format
from pathlib import Path
import os
import bibtexparser
from numpy.typing import NDArray
from typing import Dict, List, Union, Optional
from mech_dot_inp_gen import *
import jinja2

In [3]:
class OptimaOutput:
    def __init__(self, job_name: Union[str, Path],
                 optima_path: Optional[Union[str, Path]] = None):

        self.job_name = str(job_name)
        if optima_path == None:
            self.optima_path = Path("/home/szupernikusz/OptimaPP/outputs")
        self.job_folder = self.optima_path / job_name
        file_path = self.job_folder / "mechanismInfo.txt"
        try:
            with open(file_path, "r") as f:
                self.mech_info = f.read()
        except Exception as e:
            raise e

In [4]:
class OptimaMechtest(OptimaOutput):
    def __init__(self, job_name: Union[str, Path],
                 input_mech: str,
                 optima_path: Optional[Union[str, Path]] = None,
                 errf_type: Union[str, List[str]] = "default"):
        super().__init__(job_name, optima_path)

        self._errf_files = {
           "default": "errfValues",
           "data_series": "errfValues_by_data_series", 
           "points": "errfValues_by_points",
           "species": "errfValues_by_species"
            }

        if (self.job_folder / "debug").exists():
            self.all_data = {}
            for csv_data in (self.job_folder / "debug").glob("*.csv"):
                try:
                    self.all_data[csv_data.stem] = pd.read_csv(csv_data)
                except Exception as e:
                    print(e)
        print((self.job_folder / f"mechTestResults_{input_mech}.csv"))
        if (self.job_folder / f"mechTestResults_{input_mech}.csv").exists():
            print((self.job_folder / f"mechTestResults_{input_mech}.csv").exists())
            self.all_sheets_dP = pd.read_csv(self.job_folder / f"mechTestResults_{input_mech}.csv",  # dP as in the info in dataPoints
                                        header=None,
                                        delimiter=';',
                                        index_col=False,
                                        names=['xml', 'time_point', 'species', 'dP_val', 'sim_val'],
                                        )#low_memory=False)
            #print(self.all_sheets_dP)
            self.get_coarse_df(all_sheets_dP=self.all_sheets_dP)

        stac_eq_df = pd.concat({k: v.iloc[-1] for k, v in self.all_data.items()}, axis=1)                # makes a dict() with the
        #followed18 = pd.concat({k: v.iloc[-1] for k, v in self.orig_time_sim_df.items()}, axis=1)        # same keys, the values are
        self.df_basal = stac_eq_df.iloc[3:-1].T                                                          # the last elements i.e.,
        #self.df_followed18 = followed18.T                                                                # t = last time_point

    def __str__(self):
        if not self.df_basal.empty and not self.df_followed18.empty:
            return f"Mech object was successfully generated with fields\ndf_basal: {self.df_basal.shape}\ndf_followed18: {self.df_followed18.shape}"
        elif self.df_basal.empty and not self.df_followed18.empty:
            return "Unsuccessful, error with df_basal"
        elif not self.df_basal.empty and self.df_followed18.empty:
            return "Unsuccessful, error with df_followed18"
        else:
            return "Mech object was not successfully created"

    def get_coarse_df(self, all_sheets_dP):
        self.orig_time_sim_df = {}
        self.orig_time_exp_df = {}

        time_point = 0
        species = 'ilyen_species_tuti_nem_lesz'
        self.failed_sims_xmls = []

        for _, row in all_sheets_dP.iterrows():
            if row.xml in self.failed_sims_xmls:
                continue

            xml_name = row.xml
            sim_val = row.sim_val
            exp_val = row.dP_val

            if row.species == species:
                time_point += 1
            else:
                species = row.species
                time_point = 1

            if sim_val != 'FAILED':
                # ensure dict entries exist
                if xml_name not in self.orig_time_sim_df:
                    self.orig_time_sim_df[xml_name] = pd.DataFrame()
                if xml_name not in self.orig_time_exp_df:
                    self.orig_time_exp_df[xml_name] = pd.DataFrame()

                # ALWAYS assign (no else:)
                self.orig_time_sim_df[xml_name].loc[time_point, species] = float(sim_val)
                self.orig_time_exp_df[xml_name].loc[time_point, species] = exp_val
            else:
                self.failed_sims_xmls.append(xml_name)

        # make sure rows are in time order and numeric
        for xml, df in self.orig_time_sim_df.items():
            self.orig_time_sim_df[xml] = df.sort_index().apply(pd.to_numeric, errors='coerce')

        # build “followed18” as last valid values per species
        followed18 = pd.concat(
            {xml: df.ffill().iloc[-1]              # or: df.apply(lambda c: c.dropna().iloc[-1])
            for xml, df in self.orig_time_sim_df.items()},
            axis=1
        )
        self.df_followed18 = followed18.T


In [5]:
class Theoretical_Ranges:
    def __init__(self, min_max_path: str, input_names: list[str], must_be_zero: list[str],
                 scaling_factor: float=1e-12, sheet_name:str = 'icranges'):
        self.name = os.path.splitext(os.path.basename(min_max_path))[0]
        self.df_ranges = pd.read_excel(min_max_path, sheet_name=sheet_name)
        self.df_ranges['value'] = self.df_ranges['value'].astype(float)
        self.df_ranges.species = [s.upper() for s in self.df_ranges.species]
        self.scaling_factor = scaling_factor
        self.df_scaled_ranges = self.df_ranges.select_dtypes(include='number') * self.scaling_factor
        self.must_be_zero = must_be_zero
        self.input_names = input_names
        self.get_input_data(input_names)
        self.bounds = self.get_bounds()
        self.get_sigmas()
        self.gen_lookuptable()

    def get_input_data(self, input_names: list[str]) -> None:
        self.inputs = {}
        for i in input_names:
            self.inputs[i] = 0.0
        self.inputs["REF"] = 1.0
        self.inputs["Insulin"] = 1e-10

        self.input_data = pd.DataFrame([
            {'species': species, 'minconc': value*1e+12, 'value': value*1e+12, 'maxconc': value*1e+12}
            for species, value in self.inputs.items()])

    def gen_lookuptable(self) -> None:
        self.lut = pd.concat([self.df_ranges, self.input_data], ignore_index=True)    # look-up-table
        self.lut['species'] = self.lut['species'].str.upper()
        print(f"LUT was created successfully. Its dimensions are: {self.lut.shape}")

    def get_bounds(self) -> dict[str, tuple[float, float]]:
        bounds = dict()
        for index, row in self.df_ranges.iterrows():
            if row.value < 0.1:
                lb = 1e-14
                ub = 1e-13
            else:
                lb = (row.value/2)*1e-12
                ub = (row.value*1.5)*1e-12
            bounds[row.species.upper()] = [lb, ub]
        print(bounds['TBID'])
        for _, row in self.input_data.iterrows():
            if row.species.upper() not in bounds.keys():
                bounds[row.species.upper()] = [row.minconc*1e-12, row.maxconc*1e-12]
        for m in self.must_be_zero:
            bounds[m.upper()] = [0, 0]
        return bounds

    def get_sigmas(self):
        self.sigmas = dict()
        for key, value in self.bounds.items():
            if key in self.must_be_zero: # this line might be unnecessary, as I think we need sigma only if the species is an output
                self.sigmas[key] = 5e-18
            if value[1] > value[0]:
                self.sigmas[key] = ((value[1]-value[0])/8)
            else:
                self.sigmas[key] = 5e-15


In [6]:
class Experiment:
    def __init__(self, data_source, # data_source: a pandas DataFrame
                 stresses: float, species_rng: Theoretical_Ranges,
                 sheet_name: str, bibtex: str = "", inputs: dict[str, float] = {}):
        self.rng = species_rng
        if isinstance(data_source, pd.DataFrame): # Ha pandas DataFrame (azaz xlsx worksheet)
            self.name = sheet_name
            self.experiment_data = data_source.copy()
        else:
            raise ValueError("data_source must be a pandas DataFrame")

        self.stresses = stresses
        self.bibtex = self.parse_bibtex(bibtex)
        self.non_species_cols = {"TIME"}
        
        self.inputs = self.get_exp_inputs(inputs)
        self.process_data()

    def parse_bibtex(self, bibtex_str):
        parser = bibtexparser.loads(bibtex_str)
        if not parser.entries:
            raise ValueError("No valid BibTeX entry found.")
        entry = parser.entries[0]  # Assume only one entry is given

        return {
            "author": entry.get("author", ""),
            "title": entry.get("title", ""),
            "journal": entry.get("journal", ""),
            "volume": entry.get("volume", ""),
            "number": entry.get("number", ""),
            "year": entry.get("year", ""),
            "doi": entry.get("doi", entry.get("url", ""))  # fallback if no DOI
        }

    def get_exp_inputs(self, inputs) -> dict[str, float]:
        for name in self.rng.input_names:
            if name not in inputs.keys():
                inputs[name] = self.rng.inputs[name]
        return inputs

    def process_data(self) -> None:
        self.experiment_data.columns = [col.upper() for col in self.experiment_data.columns]
        self.experiment_data.rename(columns={'TIME': 'time'}, inplace=True)
        self.experiment_data.time = self.experiment_data.time #* 60  # Converting hrs to mins - if I convert, some xmls fail for some reaseon
        self.experiment_data = self.experiment_data.dropna()
        self.species = [v for v in self.experiment_data.columns if v.upper() not in self.non_species_cols and "STD" not in v.upper()]

    def quantitated_exp_data(self, ics: dict[str, float]) -> None:
        quant_Data = self.experiment_data.copy()
        #species_and_std = [col for col in quant_Data.columns if col.upper() not in self.non_species_cols]
        #for s in species_and_std:
        #    if 'STD' in s.upper():
        #        quant_Data[s] *= ics[s[0:-3]]
        #    #elif f"{s}_STD" not in species_and_std:
        #    else:
        #        quant_Data[s] *= ics[s]

        for col in quant_Data.columns:
            if col.upper() not in self.non_species_cols:
                if 'STD' in col.upper():
                    quant_Data[col] *= ics[col[0:-4].upper()]
                else:
                    quant_Data[col] *= ics[col.upper()]
                if 'STD' not in col.upper() and f"{col}_STD" not in quant_Data.columns:
                    # add a new column called f"{col}_STD" filled with self.rng.sigmas[col.upper()]
                    quant_Data[f"{col}_STD"] = self.rng.sigmas[col.upper()]

        self.quant_data = quant_Data

    def check_compatibility(self) -> None:
        for s in self.species:
            if s.upper() not in self.rng.bounds.keys():
                self.rng.bounds[s] = [0, 0]
                print(f"Creating new entry for {s}\n")
            #else:
            #    print('All compatible\n')


In [27]:
class Genesis:
    def __init__(self, experiment: Experiment, max_digit = 4, prefix: str = "stressful_life"):
        self.experiment = experiment
        self.maxdigit = max_digit
        self.prefix = prefix

    def create_xml_files(self, output_xmls_path: str, num_xmls: int, xml_template_path: str) -> None:
        self.output_dir = output_xmls_path
        if not os.path.exists(output_xmls_path):
            os.makedirs(output_xmls_path)
        self.num_xmls = num_xmls
        env = jinja2.Environment(loader=jinja2.FileSystemLoader(os.path.dirname(xml_template_path)))
        self.template = env.get_template(os.path.basename(xml_template_path))

        for i in range(1, num_xmls+1):
            np.random.seed(i)
            self.random_ics = self.get_random_ics()
            self.experiment.quantitated_exp_data(ics=self.random_ics)
            self.make_xml_output(i, output_xmls_path)

    def get_random_ics(self) -> dict[str, float]:
        random_ics = {}
        for key in self.experiment.rng.bounds.keys():
            if key not in self.experiment.rng.must_be_zero:
                random_ics[key] = np.random.uniform(*self.experiment.rng.bounds[key])
            else:
                random_ics[key] = 0
        for key in self.experiment.inputs.keys():
            random_ics[key] = self.experiment.inputs[key]
        random_ics["REF"] = 1.0
        return random_ics

    def make_xml_output(self, file_index: int, output_xmls_path: str) -> None:
        dataPoints = [self.compileDataRow(row.values) for _, row in self.experiment.quant_data.iterrows()]
        self.datP_check = dataPoints
        output = self.template.render(ics=self.random_ics, variables=self.experiment.species,
                                      dataPoints=dataPoints, bib=self.experiment.bibtex)
        padded_number = str(file_index).zfill(self.maxdigit)
        filename = f"{self.prefix}_{padded_number}.xml"
        with open(os.path.join(output_xmls_path, filename), 'w') as f:
            f.write(output)

    def compileDataRow(self, dataPoints):
        meas = "".join(f"<{v}>{{:.4e}}</{v}>" for v in self.experiment.quant_data.columns)
        return f"<dataPoint>{meas.format(*dataPoints)}</dataPoint>"


In [34]:
class Simulation:
    def __init__(self, gen: Genesis, xmls_in_one_opp: int) -> None:
      self.gen = gen
      self.get_xml_vec(xmls_in_one_opp)

    def get_xml_vec(self, xmls_in_one_opp) -> None:
      self.xmls = []
      for i in range(1, self.gen.num_xmls, xmls_in_one_opp):
          xml_cnt = np.arange(i, i+xmls_in_one_opp, 1) 
          self.xmls.append(xml_cnt)

    def opp_gen(self, opp_output_dir, opp_name_prefix, kiir, mech_file, food) -> None:
      self.opps = []
      self.indices = []
      self.food = food
      self.mech_file = mech_file
      self.mech_inp_name = self.mech_file.split('/')[-1].split('.inp')[0]
      for num in self.xmls:
          opp_filename = f"{opp_name_prefix}_{num[-1]}.opp"
          self.opps.append(opp_filename)
          self.indices.append(f"{num[-1]}")
          if kiir:
            opp_content = self.generate_opp_content(xml_folder=self.gen.output_dir,
                                                    num_xmls=num,
                                                    mech_file=mech_file,
                                                    name=self.gen.prefix)
            with open(os.path.join(opp_output_dir, opp_filename), "w") as f:
              f.write(opp_content)

    def generate_opp_content(self, xml_folder: str, num_xmls: Union[list[int], list[list[int]]],
                             mech_file: str, name: str, time_limit: int = 50, thread_limit: int = 32,
                             settings_tag: str = "systems_biology", solver: str = "cantera") -> str:
      mech_name = mech_file.split('/')[-1].split('.inp')[0]
      yaml = mech_file.split('.inp')[0]

      # Create MECHMOD section
      mechmod = f"""MECHMOD
      USE_NAME         {mech_name}
      MECH_FILE        {mech_file}
      COMPILE_{solver} {yaml}.yaml
      END
      """

      # Create MECHTEST section
      mechtest = f"""MECHTEST
      MECHANISM  {mech_name}
      TIME_LIMIT {time_limit}
      THREAD_LIMIT {thread_limit}
      SETTINGS_TAG {settings_tag}
      FALLBACK_TO_DEFAULT_SETTINGS
      PLOTS FALSE

      SOLVER {solver}
      SAVE_STATES      CSV
      """

      # Add each XML file name
      for xml in num_xmls:
          padded_number = str(xml).zfill(self.gen.maxdigit)
          mechtest += f"      NAME {xml_folder}/{name}_{padded_number}.xml\n"

      mechtest += "END\n"

      return mechmod + "\n" + mechtest

    @staticmethod
    def generate_opp_content_from_paths(xml_paths: list[str], mech_file: str,
                                        time_limit: int = 50, thread_limit: int = 32,
                                        settings_tag: str = "systems_biology", solver: str = "cantera") -> str:
      mech_name = mech_file.split('/')[-1].split('.inp')[0]
      yaml = mech_file.split('.inp')[0]

      # Create MECHMOD section
      mechmod = f"""MECHMOD
      USE_NAME         {mech_name}
      MECH_FILE        {mech_file}
      COMPILE_{solver} {yaml}.yaml
      END
      """

      # Create MECHTEST section
      mechtest = f"""MECHTEST
      MECHANISM  {mech_name}
      TIME_LIMIT {time_limit}
      THREAD_LIMIT {thread_limit}
      SETTINGS_TAG {settings_tag}
      FALLBACK_TO_DEFAULT_SETTINGS
      PLOTS FALSE

      SOLVER {solver}
      SAVE_STATES      CSV
      """

      # Add each XML file name
      for xml_path in xml_paths:
          mechtest += f"      NAME {xml_path}\n"

      mechtest += "END\n"

      return mechmod + "\n" + mechtest

    def sim_runner(self, log_location:str = ''):
      self.parent_path = Path.cwd().parents[2]

      if log_location == '':
        for idx, opp_file in enumerate(self.opps):
            command = ["bin/Release/OptimaPP", f"7_Krisztian/1_mechtest/{opp_file}"]
            print(f"Running: {' '.join(command)}")
            subprocess.run(command, check=True, cwd=self.parent_path)
      else:
        for idx, opp_file in enumerate(self.opps):
          command = ["bin/Release/OptimaPP", f"7_Krisztian/1_mechtest/{opp_file}"]
          print(f"Running: {' '.join(command)}")
          log_idx = self.xmls[idx][-1]
          with open(f"{log_location}/run_log_stac_starve_rap{log_idx}.txt", "w") as log:
              subprocess.run(command, check=True, stdout=log, stderr=subprocess.STDOUT, cwd=self.parent_path)


In [10]:
# 1e-12 scaling for mol/cm^3
# uniformly dist. rap of [rap_input] nano mol     NOT     mol/dm^3 (= nM) cc.
# in a volume of V = 10e-12 dm^3
# uniformly "absorbed" by 2*10^5 cells
rap_input = np.array([100, 50, 10, 5, 1, 0.5, 0.1, 0]) * 1e-12  # Converting to mol/cm^3 (=mol/mL)
rap_in_cells = rap_input / (200000 * 1e-12 * 1e+3)  # rap_in_well_mol / (cell_num * V_cell_in_L * conversion_factor_to_mL)
stress1 = 'rap'
stresses: dict[str, NDArray[np.float64]] = {stress1: rap_in_cells}
stresses

{'rap': array([5.0e-07, 2.5e-07, 5.0e-08, 2.5e-08, 5.0e-09, 2.5e-09, 5.0e-10,
        0.0e+00])}

In [11]:
input_names = ['RAPOUT', 'TG', 'CCH', 'REF', 'Insulin', 'TG_SERCA', 'RKMTORA', 'casp', 'IP3R', 'Baxa', 'tBid']
must_be_zero = ['casp', 'Baxa', 'tBid', 'p53a', 'PUMA', 'RAP', 'RK']

rng = Theoretical_Ranges('../input_files/theoretical_combined_ranges.xlsx',
                         input_names, must_be_zero, sheet_name='combined_ranges')

[1e-14, 1e-13]
LUT was created successfully. Its dimensions are: (121, 4)


In [12]:
# FKBP12 set to MTORA ranges
print(rng.bounds['FKBP12'], rng.bounds['MTORA'])

[1.875e-10, 5.625e-10] [1.875e-10, 5.625e-10]


In [43]:
exp_xlsx_path = '../input_files/Nitin_rap.xlsx'
xml_template = '../input_files/xml_template.xml'

# Parse stresses

# Read all sheets from Excel
all_sheets = pd.read_excel(exp_xlsx_path, sheet_name=None)  # dict of {sheet_name: DataFrame}

# Extract BibTeX from the last sheet
last_sheet_name = list(all_sheets.keys())[-1]
bibtex_df = all_sheets[last_sheet_name]
# Ha nem lenne header a BibTex-nel, akk ezzel kell beolvasni a sheetet: bibtex_df = pd.read_excel(exp_xlsx_path, sheet_name=last_sheet_name, header=None)

# Join all non-empty strings from the first column into a BibTeX string
bibtex_lines = bibtex_df.iloc[:, 0].dropna().astype(str).tolist()
bibtex_str = "\n".join(bibtex_lines)
bibtex_str = "\n".join(bibtex_lines).replace("_x000d_", "")  # Clean malformed carriage returns

#print("BibTex:\n", bibtex_str, "\n")
opp_output_dir = '../../1_mechtest'
num_xmls = 10

all_xml_paths = []

date = datetime.datetime.now()
date_prefix = f"{date.year}{date.month}{date.day}"

for i, sheet_name in enumerate(list(all_sheets.keys())[:-1]):
    df = all_sheets[sheet_name]
    exp = Experiment(df, stresses['rap'][i], rng, sheet_name, bibtex_str, inputs={'RAPOUT': 1e-10})
    output_dir = f"/home/szupernikusz/Projects/Semmelweis_BCRN/BCRN/xml/{exp.bibtex['author'].split()[0][:-1]}_{exp.bibtex['year']}/{exp.name}"
    gen = Genesis(exp, max_digit=4, prefix=f"alma{exp.name}")
    gen.create_xml_files(output_dir, num_xmls, xml_template)

    for xml in range(1, num_xmls + 1):
        padded_number = str(xml).zfill(gen.maxdigit)
        all_xml_paths.append(f"{output_dir}/{gen.prefix}_{padded_number}.xml")

opp_filename = f"{date_prefix}_all_setups.opp"
opp_content = Simulation.generate_opp_content_from_paths(
    xml_paths=all_xml_paths,
    mech_file='/home/szupernikusz/Projects/Semmelweis_BCRN/BCRN/mech/starve_rap_combined.inp',
)

with open(os.path.join(opp_output_dir, opp_filename), "w") as f:
    f.write(opp_content)


In [70]:
exp.quant_data

,time,CYTC,CYTC_STD
0,0,3.50e-15,1.13e-14
1,8,2.33e-14,1.13e-14
2,22,5.28e-14,1.13e-14


In [71]:
# unique_species.py

# Read the input file
with open("../input_files/species.txt", "r") as f:
    text = f.read()

# Split on any whitespace and create a set
species_set = set(text.split())

# (Optional) sort for nicer output
unique_species = sorted(species_set)

# Print results
print(f"Number of unique species: {len(unique_species)}")

# (Optional) save to a file
with open("../input_files/unique_species.txt", "w") as f:
    for s in unique_species:
        f.write(s + "\n")


Number of unique species: 114


In [28]:
write_inp(list(species_set), out_path='../../mech/dummy.inp')

In [43]:
bcrn6_ranges = pd.read_csv('../input_files/BCRN6_ranges.csv')
gluc_rap_ranges = pd.read_csv('../input_files/gluc_rap_ranges.csv')

In [44]:
bcrn6_ranges.head()

,species,minconc,value,maxconc
0,AC,1.00e+02,200,400
1,AC_Ga,0.00e+00,0,0
2,AC_Gaa,0.00e+00,0,0
3,AKT,0.00e+00,0,0
4,AKTa,5.00e+01,100,200


In [45]:
gluc_rap_ranges['minconc'] = gluc_rap_ranges['minconc'].astype(float) * 1e+12
gluc_rap_ranges['value'] = gluc_rap_ranges['value'].astype(float) * 1e+12
gluc_rap_ranges['maxconc'] = gluc_rap_ranges['maxconc'].astype(float) * 1e+12
gluc_rap_ranges.head()

,species,minconc,value,maxconc
0,PYRUVATE,2.00e+04,3.30e+04,4.00e+04
1,Q,1.00e+03,1.00e+03,1.00e+03
2,ATP,3.00e+06,5.00e+06,7.00e+06
3,ADP,5.00e+04,1.25e+05,2.00e+05
4,AMP,5.00e+02,2.50e+03,5.00e+03


In [46]:
combined_ranges = gluc_rap_ranges.copy()
for index, row in bcrn6_ranges.iterrows():
    if row['species'] not in gluc_rap_ranges['species'].values:
        combined_ranges = pd.concat([combined_ranges, pd.DataFrame([row])], ignore_index=True)
combined_ranges

,species,minconc,value,maxconc
0,PYRUVATE,2.00e+04,3.30e+04,4.00e+04
1,Q,1.00e+03,1.00e+03,1.00e+03
2,ATP,3.00e+06,5.00e+06,7.00e+06
3,ADP,5.00e+04,1.25e+05,2.00e+05
4,AMP,5.00e+02,2.50e+03,5.00e+03
...,...,...,...,...
105,preAUT,0.00e+00,0.00e+00,0.00e+00
106,procasp,1.20e+01,2.40e+01,4.80e+01
107,tBid,0.00e+00,0.00e+00,0.00e+00
108,PKC,5.00e+01,1.00e+02,2.00e+02


In [47]:
combined_species = combined_ranges['species'].str.upper().tolist()
set(combined_species) - species_set

set()

In [48]:
combined_ranges.to_csv('../input_files/combined_ranges.csv', index=False)

In [39]:
species_set
# I have to be in the folder where 'bin' is, or rewrite the commands below!!!!!!

# bin/Release/OptimaPP 7_Krisztian/1_mechtest/2025625_BCRN_Beesabathuni_Rap_100_nM.opp

{'AC',
 'AC_GA',
 'AC_GAA',
 'ADP',
 'AKT',
 'AKTA',
 'AMP',
 'AMPK',
 'AMPKA',
 'AMPKA_ADP',
 'AMPKA_ADP_PP2A',
 'AMPKA_AMP',
 'AMPKA_AMP_PP2A',
 'AMPKA_ATP',
 'AMPKA_ATP_PP2A',
 'AMPKA_PP2A',
 'AMPK_ADP',
 'AMPK_ADP_LKB1',
 'AMPK_AMP',
 'AMPK_AMP_LKB1',
 'AMPK_ATP',
 'AMPK_ATP_LKB1',
 'AMPK_LKB1',
 'ATG5',
 'ATG5T',
 'ATG5_BCL2',
 'ATP',
 'AUT',
 'BAX',
 'BAXA',
 'BCL2',
 'BCL2_BAX',
 'BCL2_PUMA',
 'BEC1',
 'BEC1A',
 'BEC1_BCL2',
 'BEC1_UVG',
 'BID',
 'CA2ER',
 'CA2IC',
 'CALPAIN',
 'CALPAINA',
 'CAMKKB',
 'CAMKKBA',
 'CAMP',
 'CASP',
 'CCH',
 'CYTC',
 'CYTCM',
 'DAPK',
 'DAPKA',
 'DRAM',
 'DS',
 'EPAC',
 'EPACA',
 'FKBP12',
 'GA',
 'GAA',
 'GABC',
 'GBC',
 'GLUCIN',
 'GLUCOUT',
 'GLUT1',
 'GLUT1_GLUCIN',
 'GLUT1_GLUCOUT',
 'GPCRA',
 'INSULIN',
 'IP3',
 'IP3R',
 'IP3R_BCL2',
 'IP3R_IP3',
 'LKB1',
 'MDM2',
 'MTOR',
 'MTORA',
 'MTOR_RAP',
 'NS',
 'P53',
 'P53A',
 'P53A_BCL2',
 'PHAG',
 'PI3K',
 'PI3KA',
 'PIP2',
 'PKA',
 'PKAA',
 'PKC',
 'PKC_CA2IC',
 'PLCE',
 'PLCEA',
 'PP2A',
 'PREAU

In [29]:
sim.gen.experiment.quant_data

,time,CYTC,CYTC_STD
0,0,3.50e-15,1.13e-14
1,8,2.33e-14,1.13e-14
2,22,5.28e-14,1.13e-14


In [30]:
gen.datP_check

['<dataPoint><time>0.0000e+00</time><CYTC>3.5037e-15</CYTC><CYTC_STD>1.1250e-14</CYTC_STD></dataPoint>',
 '<dataPoint><time>8.0000e+00</time><CYTC>2.3289e-14</CYTC><CYTC_STD>1.1250e-14</CYTC_STD></dataPoint>',
 '<dataPoint><time>2.2000e+01</time><CYTC>5.2762e-14</CYTC><CYTC_STD>1.1250e-14</CYTC_STD></dataPoint>']